# Load and Enrich multiple file types Azure Cognitive Search

이 단계에서는 지정된 Azure Blob Storage의 데이터를 로드하고 Skillset을 생성합니다. Azure Cognitive Search에서 로드된 데이터의 정보를 분석하고 추출하는 Skillset을 사용하여 이미지 및 응용 프로그램 파일과 같은 Azure Storage의 혼합 콘텐츠에 대한 작업을 수행합니다. 

이 데모에서는 PDF형식의 뉴스 출판물이 들어있는 Arxiv라는 Azure Blob Storage Container를 사용할 예정입니다. 

현재 데모에서는 PDF 파일만 사용되지만 훨씬 더 큰 규모로 수행할 수 있으며, Azure Cognitive Search는 Microsoft Office(DOCX/DOC, XSLX/XLS, PPTX/PPT, MSG), HTML, XML, ZIP 및 일반 텍스트 파일(JSON 포함) 등 다양한 파일 형식을 지원합니다. 

이번 notebook에서는 다음과 같은 Cognitive Search의 기능을 만듭니다:

+ Skillset
+ Search index
+ Indexer

이 notebook을 실행하려면 README에서 Azure 서비스를 이미 생성한 상태여야 하며 이 작업을 통해 인덱서가 완료되고 검색 인덱스가 로드될 때까지 쿼리는 결과를 반환하지 않습니다. 

![cog-search](https://raw.githubusercontent.com/MSUSAzureAccelerators/Azure-Cognitive-Search-Azure-OpenAI-Accelerator/69114f93a7c245fd79009f01169e746859776d77//images/Cog-Search-Enrich.png)

In [2]:
import os
import json
import requests
from dotenv import load_dotenv
load_dotenv("credentials.env")

# credentials.env 파일안에 연결된 Blob 스토리지 컨테이너 이름
BLOB_CONTAINER_NAME = "arxivcs"

In [3]:
# 데이터 소스, 스킬셋, 인덱스 및 인덱서 이름 정의합니다. 
datasource_name = "cogsrch-datasource-files"
skillset_name = "cogsrch-skillset-files"
index_name = "cogsrch-index-files"
indexer_name = "cogsrch-indexer-files"

In [4]:
# Payloads header를 설정합니다. 
headers = {'Content-Type': 'application/json','api-key': os.environ['AZURE_SEARCH_KEY']}
params = {'api-version': os.environ['AZURE_SEARCH_API_VERSION']}

## 데이터 소스 생성

In [5]:
# Json Payload를 Azure Search 엔진으로 보내는 데이터 소스를 만듭니다. 

datasource_payload = {
    "name": datasource_name,
    "description": "Demo files to demonstrate cognitive search capabilities.",
    "type": "azureblob",
    "credentials": {
        "connectionString": os.environ['BLOB_CONNECTION_STRING']
    },
    # "dataDeletionDetectionPolicy" : {
    #     "@odata.type" :"#Microsoft.Azure.Search.NativeBlobSoftDeleteDeletionDetectionPolicy" # this makes sure that if the item is deleted from the source, it will be deleted from the index
    # },
    "container": {
        "name": BLOB_CONTAINER_NAME
    }
}
r = requests.put(os.environ['AZURE_SEARCH_ENDPOINT'] + "/datasources/" + datasource_name,
                 data=json.dumps(datasource_payload), headers=headers, params=params)
print(r.status_code)
print(r.ok)

201
True


- 201 - 성공
- 204 - 덮어쓰기 성공
- 40X - 권한 에러 

만약 403 error가 나온다면 endpoint나 key을 확인을 해주세요.

## Skillset 생성 - OCR, Text Splitter, Language Detection, KeyPhrase extraction, Entity Recognition

이번 단계에서는 Skillset을 만듭니다. Skillset은 많은 Cognitive Services를 사용하여 문서에서 정보를 추출하고 OCR을 적용하여 번역하는 등 문서를 풍부하게 만드는 단계입니다.

https://learn.microsoft.com/ko-kr/azure/search/cognitive-search-working-with-skillsets

https://learn.microsoft.com/ko-kr/azure/search/cognitive-search-predefined-skills


In [6]:
# 스킬셋 생성
skillset_payload = {
    "name": skillset_name,
    "description": "Extract entities, detect language and extract key-phrases",
    "skills":
    [
        {
            "@odata.type": "#Microsoft.Skills.Vision.OcrSkill",
            "description": "Extract text (plain and structured) from image.",
            "context": "/document/normalized_images/*",
            "defaultLanguageCode": "en",
            "detectOrientation": True,
            "inputs": [
                {
                  "name": "image",
                  "source": "/document/normalized_images/*"
                }
            ],
                "outputs": [
                {
                  "name": "text",
                  "targetName" : "images_text"
                }
            ]
        },
        {
            "@odata.type": "#Microsoft.Skills.Text.MergeSkill",
            "description": "Create merged_text, which includes all the textual representation of each image inserted at the right location in the content field. This is useful for PDF and other file formats that supported embedded images.",
            "context": "/document",
            "insertPreTag": " ",
            "insertPostTag": " ",
            "inputs": [
                {
                  "name":"text", "source": "/document/content"
                },
                {
                  "name": "itemsToInsert", "source": "/document/normalized_images/*/images_text"
                },
                {
                  "name":"offsets", "source": "/document/normalized_images/*/contentOffset"
                }
            ],
            "outputs": [
                {
                  "name": "mergedText", 
                  "targetName" : "merged_text"
                }
            ]
        },
        {
            "@odata.type": "#Microsoft.Skills.Text.LanguageDetectionSkill",
            "context": "/document",
            "description": "If you have multilingual content, adding a language code is useful for filtering",
            "inputs": [
                {
                  "name": "text",
                  "source": "/document/content"
                }
            ],
            "outputs": [
                {
                  "name": "languageCode",
                  "targetName": "language"
                }
            ]
        },
        {
            "@odata.type": "#Microsoft.Skills.Text.SplitSkill",
            "context": "/document",
            "textSplitMode": "pages",
            "maximumPageLength": 5000, # 5000 is default
            "defaultLanguageCode": "en",
            "inputs": [
                {
                    "name": "text",
                    "source": "/document/merged_text"
                },
                {
                    "name": "languageCode",
                    "source": "/document/language"
                }
            ],
            "outputs": [
                {
                    "name": "textItems",
                    "targetName": "pages"
                }
            ]
        },
        {
            "@odata.type": "#Microsoft.Skills.Text.KeyPhraseExtractionSkill",
            "context": "/document/pages/*",
            "maxKeyPhraseCount": 2,
            "defaultLanguageCode": "en",
            "inputs": [
                {
                    "name": "text", 
                    "source": "/document/pages/*"
                },
                {
                    "name": "languageCode",
                    "source": "/document/language"
                }
            ],
            "outputs": [
                {
                    "name": "keyPhrases",
                    "targetName": "keyPhrases"
                }
            ]
        },
        {
            "@odata.type": "#Microsoft.Skills.Text.V3.EntityRecognitionSkill",
            "context": "/document/pages/*",
            "categories": ["Person", "URL", "Email"],
            "minimumPrecision": 0.5, 
            "defaultLanguageCode": "en",
            "inputs": [
                {
                    "name": "text", 
                    "source":"/document/pages/*"
                },
                {
                    "name": "languageCode",
                    "source": "/document/language"
                }
            ],
            "outputs": [
                {
                    "name": "persons", 
                    "targetName": "persons"
                },
                {
                    "name": "urls", 
                    "targetName": "urls"
                },
                {
                    "name": "emails", 
                    "targetName": "emails"
                }
            ]
        }
    ],
    "cognitiveServices": {
        "@odata.type": "#Microsoft.Azure.Search.CognitiveServicesByKey",
        "description": os.environ['COG_SERVICES_NAME'],
        "key": os.environ['COG_SERVICES_KEY']
    }
}

r = requests.put(os.environ['AZURE_SEARCH_ENDPOINT'] + "/skillsets/" + skillset_name,
                 data=json.dumps(skillset_payload), headers=headers, params=params)
print(r.status_code)
print(r.ok)

201
True


## Index 생성

Azure Cognitive Search에서 검색 인덱스는 검색 엔진에서 인덱싱, 전체 텍스트 검색 및 필터링된 쿼리에 사용할 수 있는 검색 기능입니다. 인덱스는 스키마로 정의되어 검색 서비스에 저장됩니다. 

아래의 코드는 검색 인덱스의 스키마를 정의합니다. 필드 컬렉션에서는 하나의 필드를 키로 지정해야 합니다.

Reference:

https://learn.microsoft.com/ko-kr/azure/search/search-what-is-an-index

In [7]:
# 인덱스 생성
# 쿼리는 인덱스에서 검색 가능한 필드와 필터링 가능한 필드에서 작동합니다
index_payload = {
    "name": index_name,
    "fields": [
        {"name": "id", "type": "Edm.String", "key": "true", "searchable": "false", "retrievable": "true", "sortable": "false", "filterable": "false","facetable": "false"},
        {"name": "title", "type": "Edm.String", "searchable": "true", "retrievable": "true", "facetable": "false", "filterable": "true", "sortable": "false"},
        {"name": "content", "type": "Edm.String", "searchable": "true", "retrievable": "true", "sortable": "false", "filterable": "false","facetable": "false"},
        {"name": "chunks","type": "Collection(Edm.String)", "searchable": "false", "retrievable": "true", "sortable": "false", "filterable": "false", "facetable": "false"},
        {"name": "language", "type": "Edm.String", "searchable": "false", "retrievable": "true", "sortable": "true", "filterable": "true", "facetable": "true"},
        {"name": "name", "type": "Edm.String", "searchable": "true", "retrievable": "true", "sortable": "false", "filterable": "false", "facetable": "false"},
        {"name": "location", "type": "Edm.String", "searchable": "false", "retrievable": "true", "sortable": "false", "filterable": "false", "facetable": "false"},
        {"name": "vectorized", "type": "Edm.Boolean", "searchable": "false", "retrievable": "true", "sortable": "false", "filterable": "false", "facetable": "false"},
        {"name": "images_text", "type": "Collection(Edm.String)", "searchable": "true", "retrievable": "true", "sortable": "false", "filterable": "false", "facetable": "false"},
        {"name": "keyPhrases", "type": "Collection(Edm.String)", "searchable": "true", "retrievable": "true", "sortable": "false", "filterable": "true", "facetable": "true"},
        {"name": "persons", "type": "Collection(Edm.String)", "searchable": "true", "retrievable": "true", "sortable": "false", "filterable": "false", "facetable": "false"},
        {"name": "urls", "type": "Collection(Edm.String)", "searchable": "false", "retrievable": "true", "sortable": "false", "filterable": "false", "facetable": "false"},
        {"name": "emails", "type": "Collection(Edm.String)", "searchable": "true", "retrievable": "true", "sortable": "false", "filterable": "true", "facetable": "false"}
        
    ],
    "semantic": {
      "configurations": [
        {
          "name": "my-semantic-config",
          "prioritizedFields": {
            "titleField": 
                {
                    "fieldName": "title"
                },
            "prioritizedContentFields": [
                {
                    "fieldName": "content"
                }
                ]
          }
        }
      ]
    }
}

r = requests.put(os.environ['AZURE_SEARCH_ENDPOINT'] + "/indexes/" + index_name,
                 data=json.dumps(index_payload), headers=headers, params=params)
print(r.status_code)
print(r.ok)


201
True


In [8]:
#print(r.text)

### Semantic Search 기능
Azure Search에는 Semantic Search라는 기능이 있습니다. 일반적인 키워드 mathching/counting이 아닌 Deep Neural Network를 이용하여 쿼리와 내용의 의미적 의미를 기반으로 결과를 찾는 검색엔진입니다. 

[공식문서](https://learn.microsoft.com/ko-kr/azure/search/semantic-search-overview)에서 더욱 자세히 확인하실 수 있습니다. 

Semantic Search는 텍스트 기반 쿼리에 대한 초기 검색 결과의 품질을 향상시키는 기능입니다. 검색 시 Semantic Search를 활성화하면 Semantic Search는 두 가지 방식으로 쿼리 실행 파이프라인을 확장합니다:

- 첫째, 초기 결과 집합에 대한 2차 순위를 추가하여 의미론적으로 가장 관련성이 높은 결과를 목록의 맨 위에 승격시킵니다.

- 둘째, 응답에서 캡션과 답변을 추출하고 반환하며, 이를 검색 페이지에 렌더링하여 사용자의 검색 경험을 향상시킬 수 있습니다.

[여기](https://learn.microsoft.com/ko-kr/azure/search/semantic-ranking)에서 더욱 자세한 내용을 확인하실 수 있습니다. 

## Indexer 생성 및 실행 - (runs the pipeline)

지금까지 생성한 세 가지 구성 요소(데이터 소스, 스킬셋, 인덱스)를 인덱서에 입력합니다. Azure Cognitive Search에서 인덱서를 만드는 것은 전체 파이프라인을 이동시키는 이벤트입니다.

In [9]:
# 인덱서 생성
indexer_payload = {
    "name": indexer_name,
    "dataSourceName": datasource_name,
    "targetIndexName": index_name,
    "skillsetName": skillset_name,
    "schedule" : { "interval" : "PT2H"}, # 데이터 소스에서 새 컨텐츠를 확인하려는 빈도
    "fieldMappings": [
        {
          "sourceFieldName" : "metadata_storage_path",
          "targetFieldName" : "id",
          "mappingFunction" : { "name" : "base64Encode" }
        },
        {
          "sourceFieldName" : "metadata_title",
          "targetFieldName" : "title"
        },
        {
          "sourceFieldName" : "metadata_storage_name",
          "targetFieldName" : "name"
        },
        {
          "sourceFieldName" : "metadata_storage_path",
          "targetFieldName" : "location"
        }
    ],
    "outputFieldMappings":
    [
        {
            "sourceFieldName": "/document/merged_text",
            "targetFieldName": "content"
        },
        {
            "sourceFieldName": "/document/pages/*",
            "targetFieldName": "chunks"
        },
        {
            "sourceFieldName" : "/document/normalized_images/*/images_text",
            "targetFieldName" : "images_text"
        },
        {
            "sourceFieldName": "/document/language",
            "targetFieldName": "language"
        },
        {
            "sourceFieldName": "/document/pages/*/keyPhrases/*",
            "targetFieldName": "keyPhrases"
        },
        {
          "sourceFieldName" : "/document/pages/*/persons/*", 
          "targetFieldName" : "persons"
        },
        {
            "sourceFieldName": "/document/pages/*/urls/*",
            "targetFieldName": "urls"
        },
        {
            "sourceFieldName": "/document/pages/*/emails/*",
            "targetFieldName": "emails"
        }
    ],
    "parameters":
    {
        "maxFailedItems": -1,
        "maxFailedItemsPerBatch": -1,
        "configuration":
        {
            "dataToExtract": "contentAndMetadata",
            "imageAction": "generateNormalizedImages"
        }
    }
}

r = requests.put(os.environ['AZURE_SEARCH_ENDPOINT'] + "/indexers/" + indexer_name,
                 data=json.dumps(indexer_payload), headers=headers, params=params)
print(r.status_code)
print(r.ok)

201
True


In [10]:
# Uncomment if you find an error
# r.text

Note: 만약 400 Error가 나온다면 Query Key가 아닌 Azure Search Management KEY를 사용하고 있는지 확인합니다.

In [14]:
# 인덱서 상태를 가져와 실행 중임을 확인합니다
r = requests.get(os.environ['AZURE_SEARCH_ENDPOINT'] + "/indexers/" + indexer_name +
                 "/status", headers=headers, params=params)
# pprint(json.dumps(r.json(), indent=1))
print(r.status_code)
print("Status:",r.json().get('lastResult').get('status'))
print("Items Processed:",r.json().get('lastResult').get('itemsProcessed'))
print(r.ok)

200
Status: success
Items Processed: 9
True


## 벡터 기반 Index 생성

**Azure Cognitive Search의 벡터 검색 기능** ([벡터검색 설명 동영상 보기](https://aka.ms/Vector_SearchSnackableVideo)).
 
Azure Cognitive Search의 벡터 검색의 장점은 Azure Cognitive Search의 다른 기능과의 통합, 다양한 Azure 데이터스토어의 모든 유형의 데이터(텍스트, 이미지, 오디오, 비디오 등)를 사용하여 단일 생성 AI 기반 애플리케이션을 알릴 수 있으며, 검색 인덱스에서 벡터 필드를 지원합니다. 또한 순수 벡터 검색, 하이브리드 검색, 단일 통합 솔루션에서 Bing에 의해 구동되는 정교한 순위 재조정 시스템을 제공합니다.

![vector-search](https://techcommunity.microsoft.com/t5/image/serverpage/image-id/489211i001E2B9B34F483C2/image-dimensions/876x416?v=v2)


**Azure Cognitive Search에서 벡터 검색의 주요 제한 사항은 다음과 같습니다:**

- 컨텐츠에 대한 별도의 벡터 임베딩을 사용하지 않으면, Azure OpenAI와 같은 서비스를 이용하여 사용자가 직접 임베딩을 제공해야 합니다.
- 벡터 모음의 대한 필드는 없기에, 꼭 벡터 기반 인덱스는 작은 문서이거나 큰 문서를 쪼갠 chunk여야 합니다.  



In [13]:
index_payload = {
    "name": index_name + "-vector",
    "fields": [
        {"name": "id", "type": "Edm.String", "key": "true", "filterable": "true" },
        {"name": "title","type": "Edm.String","searchable": "true","retrievable": "true"},
        {"name": "chunk","type": "Edm.String","searchable": "true","retrievable": "true"},
        {"name": "chunkVector","type": "Collection(Edm.Single)","searchable": "true","retrievable": "true","dimensions": 1536,"vectorSearchConfiguration": "vectorConfig"},
        {"name": "name", "type": "Edm.String", "searchable": "true", "retrievable": "true", "sortable": "false", "filterable": "false", "facetable": "false"},
        {"name": "location", "type": "Edm.String", "searchable": "false", "retrievable": "true", "sortable": "false", "filterable": "false", "facetable": "false"},

    ],
    "vectorSearch": {
        "algorithmConfigurations": [
            {
                "name": "vectorConfig",
                "kind": "hnsw"
            }
        ]
    },
    "semantic": {
        "configurations": [
            {
                "name": "my-semantic-config",
                "prioritizedFields": {
                    "titleField": {
                        "fieldName": "title"
                    },
                    "prioritizedContentFields": [
                        {
                            "fieldName": "chunk"
                        }
                    ],
                    "prioritizedKeywordsFields": []
                }
            }
        ]
    }
}

r = requests.put(os.environ['AZURE_SEARCH_ENDPOINT'] + "/indexes/" + index_name + "-vector",
                 data=json.dumps(index_payload), headers=headers, params=params)
print(r.status_code)
print(r.ok)

201
True


# References

- https://learn.microsoft.com/en-us/azure/search/cognitive-search-tutorial-blob
- https://github.com/Azure-Samples/azure-search-python-samples/blob/main/Tutorial-AI-Enrichment/PythonTutorial-AzureSearch-AIEnrichment.ipynb
- https://github.com/Azure/azure-sdk-for-python/tree/main/sdk/search/azure-search-documents/samples
- https://learn.microsoft.com/en-us/azure/search/search-get-started-python
- https://github.com/Azure-Samples/azure-search-python-samples/blob/main/Tutorial-AI-Enrichment/PythonTutorial-AzureSearch-AIEnrichment.ipynb